In [1]:
# Imports
import os
import time
import numpy as np
import pandas as pd

from skimage import io
from skimage.transform import AffineTransform, warp, resize
from skimage.registration import phase_cross_correlation
import tifffile as tiff

# For parallel + progress bar
from joblib import Parallel, delayed
from tqdm import tqdm
from tqdm_joblib import tqdm_joblib


##############################
# Helper Functions
##############################

def apply_transform_with_padding(image, tform, output_shape, padding_value=0):
    """
    Apply the transformation with padding.
    """
    transformed_image = warp(
        image,
        inverse_map=tform.inverse,
        output_shape=output_shape,
        mode='constant',
        cval=padding_value
    )
    return transformed_image

def resize_image(image, target_shape):
    """
    Resize image to match another image's shape, if necessary.
    """
    from skimage.transform import resize
    return resize(
        image,
        target_shape,
        mode='reflect',
        anti_aliasing=True,
        preserve_range=True,
        order=1  # Try order=1 or 3. Higher order can be slower but more accurate.
    )

def save_registered_image_with_padding(output_dir, filename, image, original_dtype):
    """
    Save the registered (warped) image with appropriate dtype.
    """
    os.makedirs(output_dir, exist_ok=True)
    out_path = os.path.join(output_dir, filename)

    # Clip and convert to original dtype if it's an integer
    if np.issubdtype(original_dtype, np.integer):
        max_val = np.iinfo(original_dtype).max
        min_val = np.iinfo(original_dtype).min
        image_clipped = np.clip(image, min_val, max_val)
        image_final = image_clipped.astype(original_dtype)
    else:
        image_final = image.astype(original_dtype)

    print(f"Saving registered image to: {out_path}")
    tiff.imwrite(out_path, image_final)

def global_phase_correlation_registration(fixed_img, moving_img):
    """
    Estimate the global shift between two images using phase cross-correlation
    and return the transformed 'moving' image plus the shift.
    """
    # Phase cross-correlation gives (shift_y, shift_x)
    shift, error, diffphase = phase_cross_correlation(fixed_img, moving_img)

    # Build an AffineTransform that translates the moving image by -shift
    # because shift is the offset needed to align moving onto fixed.
    # If shift=(dy, dx), we want to move the moving image by -dy, -dx
    tform = AffineTransform(translation=(shift[1], shift[0]))

    # Warp the moving image
    registered_img = apply_transform_with_padding(
        moving_img,
        tform,
        output_shape=fixed_img.shape,
        padding_value=0
    )

    return registered_img, shift


##############################
# Settings
##############################
base_dir = "L:/Easin/OPS_Nov_2024/TIFF/Rounds"
output_dir = os.path.join(base_dir, "Registered_Images_PhaseCorr")
os.makedirs(output_dir, exist_ok=True)

tile_list = range(440)      # e.g., 0..439
round_list = range(1, 9)    # e.g., 1..8

##############################
# Process One Tile
##############################
def process_one_tile_phase(tile_num):
    tile_str = f"{tile_num:03d}"
    local_log = []

    # Fixed image is Phenotype_DAPI
    fixed_filename = f"Phenotype_Tile-{tile_str}_c0_DAPI.tif"
    fixed_path = os.path.join(base_dir, fixed_filename)
    if not os.path.exists(fixed_path):
        print(f"[Tile {tile_str}] Missing Phenotype DAPI: {fixed_path}")
        return local_log

    # Read the fixed image
    fixed_img = io.imread(fixed_path)
    dtype_fixed = fixed_img.dtype
    fixed_shape = fixed_img.shape

    # For each round's DAPI
    for round_num in round_list:
        round_str = f"ISS-Round-{round_num}"
        moving_filename = f"{round_str}_Tile-{tile_str}_c0_DAPI.tif"
        moving_path = os.path.join(base_dir, moving_filename)

        if not os.path.exists(moving_path):
            print(f"[Tile {tile_str}, Round {round_num}] Missing DAPI: {moving_path}")
            continue

        moving_img = io.imread(moving_path)
        dtype_moving = moving_img.dtype

        # Resize if shapes differ
        if moving_img.shape != fixed_shape:
            moving_img = resize_image(moving_img, fixed_shape)

        # Register with phase correlation
        registered_dapi_float, shift = global_phase_correlation_registration(
            fixed_img,
            moving_img
        )

        print(f"[Tile {tile_str}, Round {round_num}] PhaseCorr shift (y, x) = {shift}")

        # Save the registered DAPI
        reg_dapi_filename = f"{round_str}_Tile-{tile_str}_c0_DAPI_RegisteredPhaseCorr.tif"
        save_registered_image_with_padding(
            output_dir,
            reg_dapi_filename,
            registered_dapi_float,
            dtype_moving
        )

        # Warp and save other channels
        channel_info = {
            1: "G-Filter",
            2: "T-Filter",
            3: "A-Filter",
            4: "C-Filter"
        }
        for ch_idx, ch_name in channel_info.items():
            ch_filename = f"{round_str}_Tile-{tile_str}_c{ch_idx}_{ch_name}.tif"
            ch_path = os.path.join(base_dir, ch_filename)
            if not os.path.exists(ch_path):
                print(f"[Tile {tile_str}, Round {round_num}] Missing channel {ch_idx}: {ch_path}")
                continue

            ch_img = io.imread(ch_path)
            dtype_ch = ch_img.dtype
            if ch_img.shape != fixed_shape:
                ch_img = resize_image(ch_img, fixed_shape)

            # Apply the same shift-based transform
            # (Use the same AffineTransform we just constructed)
            # We can either re-build the transform from shift or re-run phase correlation for each channel.
            # Usually, we apply the same transform as the DAPI.
            # Let's be consistent: apply the same transform from DAPI.

            # Re-build the same transform from the shift
            from skimage.transform import AffineTransform
            tform_channel = AffineTransform(translation=(-shift[1], -shift[0]))
            registered_ch_float = apply_transform_with_padding(
                ch_img,
                tform_channel,
                output_shape=fixed_shape,
                padding_value=0
            )

            out_ch_filename = f"{round_str}_Tile-{tile_str}_c{ch_idx}_{ch_name}_registered_PhaseCorr.tif"
            save_registered_image_with_padding(
                output_dir,
                out_ch_filename,
                registered_ch_float,
                dtype_ch
            )

        # Log the result
        local_log.append({
            "tile": tile_str,
            "round": round_num,
            "fixed_path": fixed_path,
            "moving_path": moving_path,
            "shift_y": float(shift[0]),
            "shift_x": float(shift[1])
        })

    return local_log


##############################
# Run in Parallel
##############################
n_jobs = 1  # Increase if you have the resources

start_time = time.time()
print("Starting phase correlation registration pipeline...")

with tqdm_joblib(tqdm(desc="Tiles", total=len(tile_list))) as progress_bar:
    all_logs = Parallel(n_jobs=n_jobs)(
        delayed(process_one_tile_phase)(tile_num) for tile_num in tile_list
    )

end_time = time.time()
print(f"Registration completed in {end_time - start_time:.2f} seconds.")

# Flatten logs
all_logs_flat = [row for partial_log in all_logs for row in partial_log]

# Save to CSV
df_log = pd.DataFrame(all_logs_flat)
log_csv = os.path.join(output_dir, "registration_phase_corr_log.csv")
df_log.to_csv(log_csv, index=False)
print(f"Registration log saved: {log_csv}")


C:\Users\syede\AppData\Roaming\Python\Python312\site-packages\tqdm_joblib\__init__.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Starting phase correlation registration pipeline...


  0%|          | 0/440 [00:00<?, ?it/s]

[Tile 000] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-000_c0_DAPI.tif
[Tile 001] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-001_c0_DAPI.tif
[Tile 002] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-002_c0_DAPI.tif
[Tile 003] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-003_c0_DAPI.tif
[Tile 004] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-004_c0_DAPI.tif
[Tile 005] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-005_c0_DAPI.tif
[Tile 006] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-006_c0_DAPI.tif
[Tile 007] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-007_c0_DAPI.tif
[Tile 008] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-008_c0_DAPI.tif
[Tile 009] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-009_c0_DAPI.tif



  5%|▌         | 24/440 [01:19<22:58,  3.31s/it]

...

[Tile 415, Round 1] PhaseCorr shift (y, x) = [  7. -15.]
Saving registered image to: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Registered_Images_PhaseCorr\ISS-Round-1_Tile-415_c0_DAPI_RegisteredPhaseCorr.tif
Saving registered image to: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Registered_Images_PhaseCorr\ISS-Round-1_Tile-415_c1_G-Filter_registered_PhaseCorr.tif
Saving registered image to: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Registered_Images_PhaseCorr\ISS-Round-1_Tile-415_c2_T-Filter_registered_PhaseCorr.tif
Saving registered image to: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Registered_Images_PhaseCorr\ISS-Round-1_Tile-415_c3_A-Filter_registered_PhaseCorr.tif
Saving registered image to: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Registered_Images_PhaseCorr\ISS-Round-1_Tile-415_c4_C-Filter_registered_PhaseCorr.tif
[Tile 415, Round 2] PhaseCorr shift (y, x) = [19. 11.]
Saving registered image to: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Registered_Images_PhaseCorr\ISS-Round-2_Tile-415_c0_DAPI_RegisteredPhaseCorr.tif
Saving reg


 95%|█████████▍| 416/440 [8:01:00<31:58, 79.95s/it]

[Tile 416, Round 1] PhaseCorr shift (y, x) = [  7. -20.]
Saving registered image to: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Registered_Images_PhaseCorr\ISS-Round-1_Tile-416_c0_DAPI_RegisteredPhaseCorr.tif
Saving registered image to: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Registered_Images_PhaseCorr\ISS-Round-1_Tile-416_c1_G-Filter_registered_PhaseCorr.tif
Saving registered image to: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Registered_Images_PhaseCorr\ISS-Round-1_Tile-416_c2_T-Filter_registered_PhaseCorr.tif
Saving registered image to: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Registered_Images_PhaseCorr\ISS-Round-1_Tile-416_c3_A-Filter_registered_PhaseCorr.tif
Saving registered image to: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Registered_Images_PhaseCorr\ISS-Round-1_Tile-416_c4_C-Filter_registered_PhaseCorr.tif
[Tile 416, Round 2] PhaseCorr shift (y, x) = [19.  8.]
Saving registered image to: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Registered_Images_PhaseCorr\ISS-Round-2_Tile-416_c0_DAPI_RegisteredPhaseCorr.tif
Saving reg


100%|██████████| 440/440 [8:02:22<00:00, 65.78s/it]

[Tile 417] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-417_c0_DAPI.tif
[Tile 418] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-418_c0_DAPI.tif
[Tile 419] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-419_c0_DAPI.tif
[Tile 420] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-420_c0_DAPI.tif
[Tile 421] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-421_c0_DAPI.tif
[Tile 422] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-422_c0_DAPI.tif
[Tile 423] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-423_c0_DAPI.tif
[Tile 424] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-424_c0_DAPI.tif
[Tile 425] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-425_c0_DAPI.tif
[Tile 426] Missing Phenotype DAPI: L:/Easin/OPS_Nov_2024/TIFF/Rounds\Phenotype_Tile-426_c0_DAPI.tif
